### Chapter 12: Assertions

SystemVerilog Assertions (SVA) provide a powerful declarative way to specify and verify design behavior. Assertions help catch bugs early, document design intent, and improve verification quality by expressing temporal relationships and protocol requirements.

#### Introduction to Assertions

Assertions are statements that specify expected behavior of a design. They can be used for:
- **Verification**: Detecting violations during simulation
- **Documentation**: Capturing design intent and protocols
- **Formal verification**: Mathematical proof of properties
- **Coverage**: Measuring verification completeness

```systemverilog
// Basic assertion syntax
assert (condition) else $error("Assertion failed");
```

#### Immediate Assertions

Immediate assertions are evaluated immediately when encountered during simulation, similar to if-statements.

##### Basic Immediate Assertions

```systemverilog
module immediate_assertions_example;
    logic clk, reset, valid, ready;
    logic [7:0] data;
    
    // Simple immediate assertion
    always_comb begin
        assert (data <= 8'hFF) 
        else $error("Data exceeds maximum value");
    end
    
    // Assertion with custom message
    always @(posedge clk) begin
        if (valid) begin
            assert (ready) 
            else $error("Ready must be high when valid is asserted at time %t", $time);
        end
    end
    
    // Assertion with severity levels
    initial begin
        assert (reset === 1'b0) 
        else $fatal("Reset must be deasserted at start");
        
        assert (clk !== 1'bx) 
        else $warning("Clock should not be unknown");
    end
endmodule
```

##### Immediate Assertion Actions

```systemverilog
module assertion_actions;
    logic [3:0] counter;
    logic enable, overflow;
    
    always @(posedge clk) begin
        // Assertion with pass and fail actions
        assert (counter < 4'hF)
            $display("Counter check passed: %d", counter);
        else begin
            $error("Counter overflow detected: %d", counter);
            overflow = 1'b1;
        end
    end
    
    // Assertion with only fail action
    always_comb begin
        assert (!overflow || !enable) 
        else $error("Enable should be low during overflow");
    end
endmodule
```

#### Concurrent Assertions

Concurrent assertions evaluate continuously over time and can express complex temporal relationships using sequences and properties.

##### Basic Concurrent Assertions

```systemverilog
module concurrent_assertions_example;
    logic clk, reset_n;
    logic req, ack, valid, ready;
    logic [7:0] data;
    
    // Simple concurrent assertion
    property req_followed_by_ack;
        @(posedge clk) req |-> ##[1:3] ack;
    endproperty
    
    assert property (req_followed_by_ack)
    else $error("Request not acknowledged within 3 cycles");
    
    // Assertion with reset handling
    property valid_data_stable;
        @(posedge clk) disable iff (!reset_n)
        valid |-> $stable(data);
    endproperty
    
    assert property (valid_data_stable);
    
    // Multiple cycle relationship
    property handshake_protocol;
        @(posedge clk) disable iff (!reset_n)
        req && !ack |=> ack within 5;
    endproperty
    
    assert property (handshake_protocol)
    else $error("Handshake protocol violation");
endmodule
```

##### Temporal Operators

```systemverilog
module temporal_operators;
    logic clk, reset_n, start, done, busy;
    logic [2:0] state;
    
    // Next cycle operator ##
    property next_cycle;
        @(posedge clk) start |-> ##1 busy;
    endproperty
    
    // Range of cycles ##[min:max]
    property completion_time;
        @(posedge clk) start |-> ##[5:10] done;
    endproperty
    
    // Eventually operator (unbounded)
    property eventually_done;
        @(posedge clk) disable iff (!reset_n)
        start |-> ##[1:$] done;
    endproperty
    
    // Throughout operator
    property busy_throughout;
        @(posedge clk) disable iff (!reset_n)
        start |-> (busy throughout ##[1:5] done);
    endproperty
    
    // Until operator
    property state_until_done;
        @(posedge clk) disable iff (!reset_n)
        start |-> (state == 3'b001) until done;
    endproperty
    
    assert property (next_cycle);
    assert property (completion_time);
    assert property (eventually_done);
    assert property (busy_throughout);
    assert property (state_until_done);
endmodule
```

#### Sequence Declarations

Sequences define temporal patterns that can be reused in multiple properties.

##### Basic Sequences

```systemverilog
module sequence_examples;
    logic clk, reset_n;
    logic valid, ready, start, done;
    logic [1:0] cmd;
    
    // Simple sequence declaration
    sequence handshake;
        @(posedge clk) valid ##1 ready;
    endsequence
    
    // Parameterized sequence
    sequence wait_cycles(int cycles);
        @(posedge clk) ##cycles 1'b1;
    endsequence
    
    // Sequence with data matching
    sequence read_cmd;
        @(posedge clk) valid && (cmd == 2'b01);
    endsequence
    
    sequence write_cmd;
        @(posedge clk) valid && (cmd == 2'b10);
    endsequence
    
    // Complex sequence with repetition
    sequence burst_transfer(int length);
        @(posedge clk) start ##1 (valid && ready)[*length] ##1 done;
    endsequence
    
    // Using sequences in properties
    property handshake_followed_by_data;
        @(posedge clk) disable iff (!reset_n)
        handshake |-> ##1 $rose(valid);
    endproperty
    
    property read_burst_4;
        @(posedge clk) disable iff (!reset_n)
        read_cmd |-> burst_transfer(4);
    endproperty
    
    assert property (handshake_followed_by_data);
    assert property (read_burst_4);
endmodule
```

##### Sequence Operators

```systemverilog
module sequence_operators;
    logic clk, a, b, c, d, valid;
    logic [7:0] data;
    
    // Concatenation
    sequence seq_concat;
        @(posedge clk) a ##1 b ##2 c;
    endsequence
    
    // Repetition operators
    sequence seq_repetition;
        @(posedge clk) a ##1 b[*3] ##1 c;  // b repeats exactly 3 times
    endsequence
    
    sequence seq_range_rep;
        @(posedge clk) a ##1 b[*2:5] ##1 c;  // b repeats 2 to 5 times
    endsequence
    
    sequence seq_goto_rep;
        @(posedge clk) a ##1 b[->3] ##1 c;  // 3 occurrences of b (not consecutive)
    endsequence
    
    sequence seq_nonconsec_rep;
        @(posedge clk) a ##1 b[=3] ##1 c;   // exactly 3 b's, last one triggers
    endsequence
    
    // Sequence intersection
    sequence seq_and;
        @(posedge clk) (a ##1 b ##1 c) and (valid throughout ##3 1);
    endsequence
    
    // Sequence union
    sequence seq_or;
        @(posedge clk) (a ##2 b) or (c ##1 d);
    endsequence
    
    // First match
    sequence first_match_seq;
        @(posedge clk) first_match(a ##[1:5] b);
    endsequence
    
    // Example properties using these sequences
    property test_concat;
        @(posedge clk) seq_concat |-> ##1 valid;
    endproperty
    
    property test_repetition;
        @(posedge clk) seq_repetition |-> ##1 (data != 8'h00);
    endproperty
    
    assert property (test_concat);
    assert property (test_repetition);
endmodule
```

#### Property Declarations

Properties combine sequences with logical and temporal operators to express complex requirements.

##### Property Structure

```systemverilog
module property_examples;
    logic clk, reset_n;
    logic req, gnt, valid, ready, busy;
    logic [3:0] id;
    
    // Basic property structure
    property basic_prop;
        @(posedge clk) disable iff (!reset_n)
        req |-> ##[1:3] gnt;
    endproperty
    
    // Property with antecedent and consequent
    property req_gnt_protocol;
        @(posedge clk) disable iff (!reset_n)
        (req && !gnt) |=> (req until gnt);
    endproperty
    
    // Property with multiple conditions
    property valid_ready_handshake;
        @(posedge clk) disable iff (!reset_n)
        valid && !ready |-> ##[1:$] (valid && ready);
    endproperty
    
    // Property with data relationships
    property id_stable_during_req;
        @(posedge clk) disable iff (!reset_n)
        req |-> $stable(id) throughout (req ##[1:$] gnt);
    endproperty
    
    // Parameterized property
    property timeout_property(int max_cycles);
        @(posedge clk) disable iff (!reset_n)
        req |-> ##[1:max_cycles] gnt;
    endproperty
    
    assert property (basic_prop);
    assert property (req_gnt_protocol);
    assert property (valid_ready_handshake);
    assert property (id_stable_during_req);
    assert property (timeout_property(10));
endmodule
```

##### Property Operators

```systemverilog
module property_operators;
    logic clk, reset_n;
    logic start, done, error, valid, ack;
    logic [2:0] state;
    
    // Implication operators
    property overlap_implication;
        @(posedge clk) disable iff (!reset_n)
        start |-> ##2 done;  // Overlapping implication
    endproperty
    
    property non_overlap_implication;
        @(posedge clk) disable iff (!reset_n)
        start |=> ##1 done;  // Non-overlapping implication
    endproperty
    
    // Logical operators in properties
    property and_property;
        @(posedge clk) disable iff (!reset_n)
        (start |-> ##2 done) and (valid |-> ##1 ack);
    endproperty
    
    property or_property;
        @(posedge clk) disable iff (!reset_n)
        (start |-> ##[1:5] done) or (error |-> ##1 reset_n);
    endproperty
    
    // Not operator
    property not_property;
        @(posedge clk) disable iff (!reset_n)
        start |-> not (error throughout ##[1:10] done);
    endproperty
    
    // If-else in properties
    property conditional_property;
        @(posedge clk) disable iff (!reset_n)
        if (state == 3'b001)
            start |-> ##[2:5] done
        else
            start |-> ##1 done;
    endproperty
    
    // Case in properties
    property case_property;
        @(posedge clk) disable iff (!reset_n)
        case (state)
            3'b001: start |-> ##2 done;
            3'b010: start |-> ##3 done;
            3'b100: start |-> ##1 done;
            default: 1'b1;  // Always true for other states
        endcase
    endproperty
    
    assert property (overlap_implication);
    assert property (non_overlap_implication);
    assert property (and_property);
    assert property (conditional_property);
    assert property (case_property);
endmodule
```

#### Assert, Assume, Cover Statements

Different types of assertions serve different purposes in verification.

##### Assert Statements

```systemverilog
module assert_statements;
    logic clk, reset_n, valid, ready, error;
    logic [7:0] data;
    
    // Basic assert
    assert property (@(posedge clk) valid |-> ready)
    else $error("Ready not asserted when valid is true");
    
    // Assert with pass action
    assert property (@(posedge clk) disable iff (!reset_n) 
                    valid |-> ##[1:3] ready)
        $display("Handshake completed successfully at time %t", $time);
    else 
        $error("Handshake timeout at time %t", $time);
    
    // Named assertion for better debugging
    property data_range_check;
        @(posedge clk) valid |-> (data inside {[0:127]});
    endproperty
    
    data_range_assertion: assert property (data_range_check)
    else $error("Data %d is out of valid range", data);
    
    // Assertion with severity control
    property no_error_during_valid;
        @(posedge clk) disable iff (!reset_n)
        valid |-> !error;
    endproperty
    
    assert property (no_error_during_valid)
    else begin
        $error("Error occurred during valid transaction");
        $finish;  // Stop simulation on critical error
    end
endmodule
```

##### Assume Statements

```systemverilog
module assume_statements;
    logic clk, reset_n, req, gnt, valid, ready;
    
    // Environment assumptions
    assume property (@(posedge clk) disable iff (!reset_n)
                    req |-> ##[1:5] !req);  // Request deasserts within 5 cycles
    
    // Input constraints for formal verification
    assume property (@(posedge clk) disable iff (!reset_n)
                    $rose(valid) |-> ##[1:3] $rose(ready));
    
    // Reset assumption
    assume property (!reset_n |=> ##[1:10] reset_n);  // Reset active for max 10 cycles
    
    // Clock assumption for formal tools
    assume property ($rose(clk) ##[1:1] $fell(clk));  // Proper clock behavior
    
    // Data stability assumption
    assume property (@(posedge clk) disable iff (!reset_n)
                    valid && !ready |=> $stable(valid));
endmodule
```

##### Cover Statements

```systemverilog
module cover_statements;
    logic clk, reset_n, start, done, error, retry;
    logic [1:0] cmd_type;
    
    // Basic coverage
    cover property (@(posedge clk) disable iff (!reset_n)
                   start ##[5:10] done);  // Cover normal completion
    
    // Corner case coverage
    cover property (@(posedge clk) disable iff (!reset_n)
                   start ##1 error ##[1:3] retry ##[2:5] done);  // Cover retry scenario
    
    // Multiple command types coverage
    cover property (@(posedge clk) disable iff (!reset_n)
                   start && (cmd_type == 2'b00));  // Read command
    
    cover property (@(posedge clk) disable iff (!reset_n)
                   start && (cmd_type == 2'b01));  // Write command
    
    cover property (@(posedge clk) disable iff (!reset_n)
                   start && (cmd_type == 2'b10));  // Special command
    
    // Back-to-back transactions
    cover property (@(posedge clk) disable iff (!reset_n)
                   start ##[3:5] done ##1 start ##[3:5] done);
    
    // Maximum delay coverage
    cover property (@(posedge clk) disable iff (!reset_n)
                   start ##10 done);  // Cover maximum delay case
    
    // Named cover for better tracking
    sequence error_recovery;
        @(posedge clk) error ##[1:2] retry ##[1:5] done;
    endsequence
    
    error_recovery_cover: cover property (error_recovery);
endmodule
```

#### Clocking and Disable Conditions

Proper clocking and reset handling are crucial for robust assertions.

##### Clocking Specification

```systemverilog
module clocking_examples;
    logic clk, fast_clk, slow_clk, reset_n;
    logic data_valid, ack, req, gnt;
    
    // Default clocking
    default clocking cb @(posedge clk);
    endclocking
    
    // Using default clocking in assertions
    property default_clock_prop;
        data_valid |-> ##2 ack;  // Uses default clocking
    endproperty
    
    // Explicit clocking
    property explicit_clock_prop;
        @(posedge fast_clk) req |-> ##[1:3] gnt;
    endproperty
    
    // Different clocks for different signals
    property cross_clock_prop;
        @(posedge clk) disable iff (!reset_n)
        data_valid |-> @(posedge fast_clk) ##[1:2] ack;
    endproperty
    
    // Negative edge clocking
    property negedge_prop;
        @(negedge clk) disable iff (!reset_n)
        req |-> ##1 $stable(gnt);
    endproperty
    
    // Dual edge clocking
    property dual_edge_prop;
        @(edge clk) disable iff (!reset_n)  // Both edges
        $changed(data_valid) |-> ##1 ack;
    endproperty
    
    assert property (default_clock_prop);
    assert property (explicit_clock_prop);
    assert property (cross_clock_prop);
    assert property (negedge_prop);
    assert property (dual_edge_prop);
endmodule
```

##### Disable Conditions

```systemverilog
module disable_conditions;
    logic clk, reset_n, soft_reset, error_mode, debug_mode;
    logic valid, ready, start, done;
    
    // Basic disable condition
    property basic_disable;
        @(posedge clk) disable iff (!reset_n)
        valid |-> ##1 ready;
    endproperty
    
    // Multiple disable conditions
    property multi_disable;
        @(posedge clk) disable iff (!reset_n || soft_reset || error_mode)
        start |-> ##[2:10] done;
    endproperty
    
    // Conditional disable
    property conditional_disable;
        @(posedge clk) disable iff (!reset_n || (debug_mode && error_mode))
        valid |-> ##1 ready;
    endproperty
    
    // Complex disable condition
    logic system_halt, maintenance_mode;
    
    property complex_disable;
        @(posedge clk) disable iff (!reset_n || 
                                   system_halt || 
                                   maintenance_mode ||
                                   (error_mode && !debug_mode))
        start |-> ##[1:5] done;
    endproperty
    
    // Disable with sequence
    sequence normal_operation;
        @(posedge clk) !error_mode && !maintenance_mode;
    endsequence
    
    property sequence_based_disable;
        @(posedge clk) disable iff (!reset_n)
        normal_operation |-> (valid |-> ##1 ready);
    endproperty
    
    assert property (basic_disable);
    assert property (multi_disable);
    assert property (conditional_disable);
    assert property (complex_disable);
    assert property (sequence_based_disable);
endmodule
```

#### Advanced Assertion Techniques

##### Local Variables in Assertions

```systemverilog
module local_variables;
    logic clk, reset_n, start, done;
    logic [7:0] data_in, data_out;
    logic [3:0] id;
    
    // Local variable to capture data
    property data_integrity;
        logic [7:0] captured_data;
        @(posedge clk) disable iff (!reset_n)
        (start, captured_data = data_in) |-> 
        ##[1:5] (done && (data_out == captured_data));
    endproperty
    
    // Local variable for ID tracking
    property id_consistency;
        logic [3:0] start_id;
        @(posedge clk) disable iff (!reset_n)
        (start, start_id = id) |->
        (id == start_id) throughout ##[1:$] done;
    endproperty
    
    // Multiple local variables
    property transaction_tracking;
        logic [7:0] req_data;
        logic [3:0] req_id;
        logic req_time;
        @(posedge clk) disable iff (!reset_n)
        (start, req_data = data_in, req_id = id, req_time = $time) |->
        ##[1:10] (done && (data_out == req_data) && (id == req_id));
    endproperty
    
    assert property (data_integrity);
    assert property (id_consistency);
    assert property (transaction_tracking);
endmodule
```

##### Assertion Subroutines

```systemverilog
module assertion_subroutines;
    logic clk, reset_n, valid, ready, error;
    logic [7:0] data;
    logic [3:0] cmd;
    
    // Function for complex data checking
    function bit valid_data_format(logic [7:0] d);
        return (d[7:4] != 4'h0) && (d[3:0] != 4'hF);
    endfunction
    
    // Function for command validation
    function bit valid_command(logic [3:0] c);
        return (c inside {4'h1, 4'h2, 4'h4, 4'h8});
    endfunction
    
    // Using functions in assertions
    property data_format_check;
        @(posedge clk) disable iff (!reset_n)
        valid |-> valid_data_format(data);
    endproperty
    
    property command_check;
        @(posedge clk) disable iff (!reset_n)
        valid |-> valid_command(cmd);
    endproperty
    
    // Task for error reporting
    task report_protocol_error(string msg, logic [7:0] d, logic [3:0] c);
        $error("%s: data=0x%h, cmd=0x%h at time %t", msg, d, c, $time);
    endtask
    
    // Complex assertion with subroutine
    property protocol_compliance;
        @(posedge clk) disable iff (!reset_n)
        valid |-> (valid_data_format(data) && valid_command(cmd));
    endproperty
    
    assert property (protocol_compliance)
    else report_protocol_error("Protocol violation", data, cmd);
    
    assert property (data_format_check);
    assert property (command_check);
endmodule
```

#### Practical Assertion Examples

##### Bus Protocol Assertions

```systemverilog
module bus_protocol_assertions;
    logic clk, reset_n;
    logic req, gnt, valid, ready, last;
    logic [31:0] addr, data;
    logic [1:0] burst_type;
    
    // Basic handshake
    property req_gnt_handshake;
        @(posedge clk) disable iff (!reset_n)
        req && !gnt |=> req until gnt;
    endproperty
    
    // Address stability during transaction
    property addr_stable;
        @(posedge clk) disable iff (!reset_n)
        req |-> $stable(addr) throughout (req ##[0:$] gnt);
    endproperty
    
    // Burst transaction
    property burst_completion;
        @(posedge clk) disable iff (!reset_n)
        gnt && (burst_type != 2'b00) |-> 
        ##1 (valid && ready)[+] ##1 last;
    endproperty
    
    // No grant without request
    property no_spurious_grant;
        @(posedge clk) disable iff (!reset_n)
        !req |-> !gnt;
    endproperty
    
    // Valid-ready protocol
    property valid_ready_protocol;
        @(posedge clk) disable iff (!reset_n)
        valid && !ready |=> valid;
    endproperty
    
    assert property (req_gnt_handshake);
    assert property (addr_stable);
    assert property (burst_completion);
    assert property (no_spurious_grant);
    assert property (valid_ready_protocol);
    
    // Coverage for different burst types
    cover property (@(posedge clk) gnt && burst_type == 2'b01);  // INCR
    cover property (@(posedge clk) gnt && burst_type == 2'b10);  // WRAP
    cover property (@(posedge clk) gnt && burst_type == 2'b11);  // FIXED
endmodule
```

##### FIFO Assertions

```systemverilog
module fifo_assertions 
    #(parameter DEPTH = 8, ADDR_WIDTH = 3)
    (
    input logic clk, reset_n,
    input logic push, pop,
    input logic [7:0] data_in,
    input logic full, empty,
    input logic [ADDR_WIDTH:0] count,
    output logic [7:0] data_out
);
    
    // FIFO never overflows
    property no_overflow;
        @(posedge clk) disable iff (!reset_n)
        full |-> !push;
    endproperty
    
    // FIFO never underflows
    property no_underflow;
        @(posedge clk) disable iff (!reset_n)
        empty |-> !pop;
    endproperty
    
    // Count consistency
    property count_max;
        @(posedge clk) disable iff (!reset_n)
        count <= DEPTH;
    endproperty
    
    // Empty and full mutual exclusion (except when DEPTH=0)
    property empty_full_mutex;
        @(posedge clk) disable iff (!reset_n)
        !(empty && full) || (DEPTH == 0);
    endproperty
    
    // Empty when count is 0
    property empty_when_zero;
        @(posedge clk) disable iff (!reset_n)
        (count == 0) |-> empty;
    endproperty
    
    // Full when count is max
    property full_when_max;
        @(posedge clk) disable iff (!reset_n)
        (count == DEPTH) |-> full;
    endproperty
    
    // Count increment on push (when not full)
    property count_increment;
        logic [ADDR_WIDTH:0] prev_count;
        @(posedge clk) disable iff (!reset_n)
        (push && !pop && !full, prev_count = count) |=>
        (count == prev_count + 1);
    endproperty
    
    // Count decrement on pop (when not empty)
    property count_decrement;
        logic [ADDR_WIDTH:0] prev_count;
        @(posedge clk) disable iff (!reset_n)
        (!push && pop && !empty, prev_count = count) |=>
        (count == prev_count - 1);
    endproperty
    
    assert property (no_overflow);
    assert property (no_underflow);
    assert property (count_max);
    assert property (empty_full_mutex);
    assert property (empty_when_zero);
    assert property (full_when_max);
    assert property (count_increment);
    assert property (count_decrement);
    
    // Coverage
    cover property (@(posedge clk) push && pop && !full && !empty);  // Simultaneous
    cover property (@(posedge clk) full);  // Full condition
    cover property (@(posedge clk) empty);  // Empty condition
endmodule
```

#### Best Practices and Guidelines

##### Assertion Writing Guidelines

```systemverilog
// GOOD: Clear, specific assertion
property good_timeout;
    @(posedge clk) disable iff (!reset_n)
    start_req |-> ##[1:MAX_TIMEOUT_CYCLES] ack_resp;
endproperty

// BAD: Vague, hard to debug
property bad_timeout;
    @(posedge clk) start_req |-> ##[1:1000] ack_resp;
endproperty

// GOOD: Named sequences for reusability
sequence valid_handshake;
    @(posedge clk) valid ##1 ready;
endsequence

property protocol_check;
    @(posedge clk) disable iff (!reset_n)
    start |-> valid_handshake;
endproperty

// GOOD: Appropriate use of local variables
property data_consistency;
    logic [31:0] saved_data;
    @(posedge clk) disable iff (!reset_n)
    (write_req, saved_data = write_data) |->
    ##[1:5] (read_req && (read_data == saved_data));
endproperty

// GOOD: Comprehensive disable conditions
property robust_assertion;
    @(posedge clk) disable iff (!reset_n || error_state || debug_mode)
    normal_operation |-> expected_response;
endproperty
```

##### Performance Considerations

```systemverilog
module performance_tips;
    logic clk, reset_n, a, b, c;
    
    // GOOD: Efficient range specification
    property efficient_range;
        @(posedge clk) disable iff (!reset_n)
        a |-> ##[1:5] b;  // Bounded range
    endproperty
    
    // BAD: Unbounded eventually (expensive)
    property expensive_eventually;
        @(posedge clk) disable iff (!reset_n)
        a |-> ##[1:$] b;  // Use sparingly
    endproperty
    
    // GOOD: Use first_match for efficiency
    property efficient_first_match;
        @(posedge clk) disable iff (!reset_n)
        a |-> first_match(##[1:10] b);
    endproperty
    
    // GOOD: Avoid complex expressions in hot paths
    logic complex_condition;
    always_comb complex_condition = (a && b) || (c && !a);
    
    property efficient_complex;
        @(posedge clk) disable iff (!reset_n)
        complex_condition |-> ##1 b;
    endproperty
    
    // BAD: Complex expression inline
    property inefficient_complex;
        @(posedge clk) disable iff (!reset_n)
        ((a && b) || (c && !a)) |-> ##1 b;
    endproperty
endmodule
```

#### Debugging Assertions

##### Assertion Debugging Techniques

```systemverilog
module assertion_debugging;
    logic clk, reset_n, req, gnt, valid, ready;
    logic [7:0] data;
    logic [3:0] state;
    
    // Add debug information to assertions
    property debug_handshake;
        @(posedge clk) disable iff (!reset_n)
        req |-> ##[1:3] gnt;
    endproperty
    
    assert property (debug_handshake)
        $display("Handshake completed: req=%b, gnt=%b at time %t", req, gnt, $time);
    else begin
        $error("Handshake failed: req=%b, gnt=%b, state=%h at time %t", 
               req, gnt, state, $time);
        $display("Additional debug info: valid=%b, ready=%b, data=%h", 
                 valid, ready, data);
    end
    
    // Use local variables for debugging
    property debug_with_locals;
        logic [7:0] captured_data;
        int start_time;
        @(posedge clk) disable iff (!reset_n)
        (valid, captured_data = data, start_time = $time) |->
        ##[1:5] (ready && (data == captured_data));
    endproperty
    
    assert property (debug_with_locals)
        $display("Data transfer OK: data=%h, duration=%0d cycles", 
                 data, ($time - start_time)/10);
    else
        $error("Data mismatch or timeout: expected=%h, got=%h, time=%0d", 
               captured_data, data, $time - start_time);
    
    // Incremental assertion building
    sequence req_phase;
        @(posedge clk) req && !gnt;
    endsequence
    
    sequence wait_phase;
        @(posedge clk) ##[1:2] 1;
    endsequence
    
    sequence gnt_phase;
        @(posedge clk) gnt;
    endsequence
    
    property incremental_debug;
        @(posedge clk) disable iff (!reset_n)
        req_phase ##0 wait_phase ##0 gnt_phase;
    endproperty
    
    // Cover intermediate steps for debugging
    cover property (@(posedge clk) req_phase);
    cover property (@(posedge clk) req_phase ##0 wait_phase);
    cover property (@(posedge clk) incremental_debug);
    
    assert property (incremental_debug);
endmodule
```

##### Assertion Reporting and Analysis

```systemverilog
module assertion_reporting;
    logic clk, reset_n, start, done, error;
    logic [15:0] transaction_count;
    
    // Custom severity levels
    `define ASSERT_ERROR(prop, msg) \
        assert property (prop) else begin \
            $error("[ASSERTION_ERROR] %s at time %t", msg, $time); \
            error_count++; \
        end
    
    `define ASSERT_WARNING(prop, msg) \
        assert property (prop) else \
            $warning("[ASSERTION_WARNING] %s at time %t", msg, $time);
    
    `define ASSERT_INFO(prop, msg) \
        assert property (prop) else \
            $info("[ASSERTION_INFO] %s at time %t", msg, $time);
    
    int error_count = 0;
    int warning_count = 0;
    
    // Usage examples
    property critical_timing;
        @(posedge clk) disable iff (!reset_n)
        start |-> ##[2:4] done;
    endproperty
    
    property performance_hint;
        @(posedge clk) disable iff (!reset_n)
        start |-> ##[1:2] done;  // Prefer faster completion
    endproperty
    
    `ASSERT_ERROR(critical_timing, "Critical timing violation")
    `ASSERT_WARNING(performance_hint, "Performance could be improved")
    
    // Assertion statistics
    always @(posedge clk) begin
        if (reset_n && done) begin
            transaction_count++;
            if (transaction_count % 1000 == 0) begin
                $display("Progress: %0d transactions completed", transaction_count);
                $display("Assertion stats: %0d errors, %0d warnings", 
                         error_count, warning_count);
            end
        end
    end
    
    // Final report
    final begin
        $display("=== ASSERTION SUMMARY ===");
        $display("Total transactions: %0d", transaction_count);
        $display("Total errors: %0d", error_count);
        $display("Total warnings: %0d", warning_count);
        if (error_count > 0) begin
            $display(" TEST FAILED with %0d assertion errors ", error_count);
        end else begin
            $display(" TEST PASSED - No assertion errors ");
        end
    end
endmodule
```

#### Formal Verification with Assertions

##### Formal-Friendly Assertions

```systemverilog
module formal_verification_example;
    logic clk, reset_n;
    logic [3:0] counter;
    logic enable, overflow;
    
    // Counter implementation
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            counter <= 4'b0;
            overflow <= 1'b0;
        end else if (enable) begin
            if (counter == 4'hF) begin
                counter <= 4'b0;
                overflow <= 1'b1;
            end else begin
                counter <= counter + 1'b1;
                overflow <= 1'b0;
            end
        end else begin
            overflow <= 1'b0;
        end
    end
    
    // Formal verification properties
    
    // Safety property: counter never exceeds maximum
    property counter_bound;
        @(posedge clk) disable iff (!reset_n)
        counter <= 4'hF;
    endproperty
    
    // Liveness property: if enabled, counter eventually reaches max
    property counter_progress;
        @(posedge clk) disable iff (!reset_n)
        enable |-> ##[1:16] (counter == 4'hF);
    endproperty
    
    // Invariant: overflow only when counter wraps
    property overflow_condition;
        @(posedge clk) disable iff (!reset_n)
        overflow |-> ($past(counter) == 4'hF && $past(enable));
    endproperty
    
    // Reset property
    property reset_behavior;
        @(posedge clk) !reset_n |=> (counter == 4'b0 && !overflow);
    endproperty
    
    // Formal directives
    assert property (counter_bound);
    assert property (overflow_condition);
    assert property (reset_behavior);
    
    // For bounded model checking
    assume property (@(posedge clk) disable iff (!reset_n) 
                    enable |-> ##[1:5] !enable);  // Bound enable duration
    
    // Cover properties for formal exploration
    cover property (@(posedge clk) counter == 4'hF);
    cover property (@(posedge clk) overflow);
    cover property (@(posedge clk) counter == 4'h0 && enable);
endmodule
```

#### Common Assertion Patterns

##### Standard Protocol Patterns

```systemverilog
module common_assertion_patterns;
    logic clk, reset_n;
    
    // Pattern 1: Request-Acknowledge
    logic req, ack;
    
    // Basic handshake
    property req_ack_basic;
        @(posedge clk) disable iff (!reset_n)
        req |-> ##[1:$] ack;
    endproperty
    
    // Persistent request
    property req_until_ack;
        @(posedge clk) disable iff (!reset_n)
        req && !ack |=> req until ack;
    endproperty
    
    // Pattern 2: Valid-Ready Protocol
    logic valid, ready;
    logic [31:0] data;
    
    // Valid-ready handshake
    property valid_ready_handshake;
        @(posedge clk) disable iff (!reset_n)
        valid && !ready |=> $stable(valid) && $stable(data);
    endproperty
    
    property ready_response;
        @(posedge clk) disable iff (!reset_n)
        valid |-> ##[0:MAX_READY_DELAY] ready;
    endproperty
    
    // Pattern 3: Credit-based flow control
    logic [3:0] credits;
    logic send, receive;
    
    property credit_conservation;
        @(posedge clk) disable iff (!reset_n)
        credits <= MAX_CREDITS;
    endproperty
    
    property no_send_without_credit;
        @(posedge clk) disable iff (!reset_n)
        send |-> (credits > 0);
    endproperty
    
    // Pattern 4: State machine assertions
    typedef enum logic [1:0] {IDLE, ACTIVE, WAIT, DONE} state_t;
    state_t current_state, next_state;
    
    property valid_state_transitions;
        @(posedge clk) disable iff (!reset_n)
        (current_state == IDLE && next_state != IDLE) |-> 
        (next_state == ACTIVE);
    endproperty
    
    property state_coverage;
        @(posedge clk) disable iff (!reset_n)
        current_state inside {IDLE, ACTIVE, WAIT, DONE};
    endproperty
    
    // Pattern 5: Mutex and resource sharing
    logic [3:0] resource_grant;
    
    property mutex_exclusive;
        @(posedge clk) disable iff (!reset_n)
        $onehot0(resource_grant);  // At most one grant
    endproperty
    
    property no_grant_without_request;
        logic [3:0] resource_req;
        @(posedge clk) disable iff (!reset_n)
        |(resource_grant) |-> |(resource_req);
    endproperty
    
    // Define MAX constants for compilation
    parameter MAX_READY_DELAY = 10;
    parameter MAX_CREDITS = 8;
    
    // Assert all patterns
    assert property (req_ack_basic);
    assert property (req_until_ack);
    assert property (valid_ready_handshake);
    assert property (ready_response);
    assert property (credit_conservation);
    assert property (no_send_without_credit);
    assert property (valid_state_transitions);
    assert property (state_coverage);
    assert property (mutex_exclusive);
    assert property (no_grant_without_request);
endmodule
```

#### Summary and Key Takeaways

SystemVerilog Assertions provide a powerful mechanism for specification and verification of design behavior. Key points to remember:

**Immediate vs Concurrent Assertions:**
- Immediate assertions: Evaluated like procedural code, good for simple checks
- Concurrent assertions: Evaluated over time, essential for temporal relationships

**Building Blocks:**
- **Sequences**: Define temporal patterns, highly reusable
- **Properties**: Combine sequences with logical relationships
- **Assertions**: Assert, assume, or cover properties

**Best Practices:**
1. Use meaningful names for assertions and properties
2. Include appropriate disable conditions (reset, error states)
3. Add informative error messages and debugging information
4. Start simple and build complex assertions incrementally
5. Use local variables to capture data for comparison
6. Consider performance implications of unbounded operators
7. Write both positive and negative test cases
8. Use cover statements to ensure assertions are exercised

**Common Applications:**
- Protocol verification (handshakes, state machines)
- Data integrity checking
- Timing constraint verification
- Resource sharing and mutex verification
- Interface compliance checking

**Tools Integration:**
- Simulation: Runtime checking and coverage collection
- Formal verification: Mathematical proof of properties
- Lint tools: Static analysis of assertion syntax and semantics

Assertions are essential for modern verification methodologies, enabling early bug detection, comprehensive coverage analysis, and formal verification. They serve as executable specifications that bridge the gap between design intent and implementation verification.